In [ ]:
import cv2
import numpy as np
import os
from abc import ABC, abstractmethod
import pickle

list of template for each cam --> dict form
naming convention: YY/MM/DD_CamID_object-to-match_level_template

In [ ]:
# naming for positions jpg: YY/MM/DD_Positions_CamID.jpg
# naming for template jpg: YY/MM/DD_CamID_ObjectToFind_level1_template.jpg

In [ ]:
class TemplateMatching(ABC):
    """
    input: directory of positions jpg for template matching (e.g. 'C:\\user\\MS\\MouseData\\positions220831\\')
            optional input: visualize_matching = Bool -> shows each plot for template matching
    output: dictionary{ camera: {object: coordinates}}, naming of the file: "results_template_matching.pickle"
    
    commands:
    
    TemplateMatching.run(filepath)
        performing template matching. results found in "results_template_matching.pickle" and
        TemplateMatching.object_coordinates_per_camera
    """
    
    @property
    def template_directory(self) ->str:
        # specifies the directory for originally used templates
        return r"C:\\Users\\MS\\Documents\\Studium\\OpeningTrack\\coding\\test_data\\templates_obj\\"
    
    @property
    def template_naming(self) ->str:
        # specifies the naming of the templates
        return 'template.jpg'
    
    @property
    def matching_naming(self) -> str:
        # specifies the naming of the jpgs to be matched
        return 'Positions.jpg'
    
    @property
    def template_matching_threshold(self) ->str:
        return 0.95
    
    @property
    def all_cameras(self) ->str:
        return ['Bottom', 'Side1', 'Side2', 'Ground1', 'Ground2', 'Top']
    
    def __init__(self, directory_positions_jpg: str, visualize_matching = False):
        
        # set path positions.jpgs from input
        self.directory_positions_jpg = directory_positions_jpg
        self.tm_methods = [cv2.TM_CCOEFF_NORMED, cv2.TM_CCORR_NORMED, cv2.TM_SQDIFF_NORMED]
        self.visualize_matching = visualize_matching
        
        
    def run(self) -> dict:    
        # create dictionary with {camera: imagepath}
        self.cameras_and_paths_positions = self._create_cameras_and_paths_dictionary(self.directory_positions_jpg, 
                                                                                     self.matching_naming)
        print('The following cameras have been found:',self.cameras_and_paths_positions.keys())

        # create dictionary {camera: {object: {level: templatepath}}}   
        self.cameras_and_paths_template = self._create_cameras_and_paths_dictionary(self.template_directory,
                                                                                    self.template_naming)
        
        # create results dictionary {camera: {(object: x-coordinate, y-coordinate)}}
        self.object_coordinates_per_camera = {}
        
        # main loop    
        for camera in self.cameras_and_paths_positions.keys():
            
            
            self.object_coordinates_per_camera[camera] = {}
            
            # load image
            original_image = cv2.imread(self.cameras_and_paths_positions[camera], 0)
            
            
            for object_of_interest in self.cameras_and_paths_template[camera].keys():
                
                
                print('Now working on ', camera, object_of_interest)
                
                # level 1
                image = original_image.copy()
                template, location = self._iterate_template_matching(camera = camera, image = image,
                                                                     level = str(1), object_of_interest = object_of_interest)

                # level 2
                ofset = location
                image = image[location[1]:location[1]+template.shape[0], location[0]:location[0]+template.shape[1]]
                template, location = self._iterate_template_matching(image = image, camera = camera,
                                                                     level = str(2), object_of_interest = object_of_interest)

                
                if '3' not in self.cameras_and_paths_template[camera][object_of_interest].keys():
                    # save results in self.object_coordinates_per_camera
                    coordinates = self._get_mean_coordinates(image, template, location)
                    transposed_coordinates = self._transpose_coordinates(coordinates, ofset)
                    self.object_coordinates_per_camera[camera][object_of_interest] = transposed_coordinates

                else:
                    # level 3
                    ofset = (ofset[0] + location[0], ofset[1]+location[1])
                    image = image[location[1]:location[1]+template.shape[0], location[0]:location[0]+template.shape[1]]
                    template, location = self._iterate_template_matching(image = image, camera = camera, 
                                                                         level = str(3), object_of_interest = object_of_interest)
                    if '4'not in self.cameras_and_paths_template[camera][object_of_interest].keys():
                        # save results in f.object_coordinates_per_camera
                        coordinates = self._get_mean_coordinates(image, template, location)
                        transposed_coordinates = self._transpose_coordinates(coordinates, ofset)
                        self.object_coordinates_per_camera[camera][object_of_interest] = transposed_coordinates
                        
                    else:
                        # level 4
                        ofset = (ofset[0] + location[0], ofset[1]+location[1])
                        image = image[location[1]:location[1]+template.shape[0], location[0]:location[0]+template.shape[1]]
                        template, location = self._iterate_template_matching(image = image, camera = camera, 
                                                                         level = str(4), object_of_interest = object_of_interest)
                        coordinates = self._get_mean_coordinates(image, template, location)
                        transposed_coordinates = self._transpose_coordinates(coordinates, ofset)
                        self.object_coordinates_per_camera[camera][object_of_interest] = transposed_coordinates
                        
        
        # saving self.object_coordinates_per_camera as object_coordinates.pickle
        with open(self.directory_positions_jpg + 'results_template_matching.pickle', 'wb') as f:
            pickle.dump(self.object_coordinates_per_camera, f, pickle.HIGHEST_PROTOCOL)
  
        
    def _create_cameras_and_paths_dictionary(self, path: str, naming_flag: str) -> dict:
        # create a dictionary for easy file loading
        
        filepaths = [path + elem for elem in os.listdir(path) if elem.endswith(naming_flag)]
        
        
        # templates
        if 'level' in filepaths[0]:
            files = {}
            cameras = []
            for filepath in filepaths:
                if filepath.endswith(naming_flag):
                    split_name = filepath.split("_")
                    camera, objecttofind, level = split_name[-4], split_name[-3], split_name[-2]
                    cameras.append(camera)
                    #objects.append(objecttofind)
                    #levels.append(level)
            for cam in cameras:
                files[cam] = {}
                objectstofind = [elem[elem.index(cam)+len(cam)+1:elem.index('_level')] for
                                      elem in os.listdir(path) if cam in elem and elem.endswith(naming_flag)]
                for objecttofind in objectstofind:
                    files[cam][objecttofind] = {}             
                    cam_filepaths = [elem for elem in filepaths if cam in elem and str(objecttofind) in elem]
                    levels = [elem[elem.index('level')+5:elem.index('level')+6] for elem in os.listdir(path) 
                              if cam in elem and str(objecttofind) in elem and elem.endswith(naming_flag)]
                    for i in range(len(levels)):
                        files[cam][objecttofind].update({levels[i]: cam_filepaths[i]})
                else:
                    continue
            
            return files
        
        # images
        else:
            # list comprehension for getting cameras
            cameras = [elem[7:(elem.index(naming_flag)-1)] for elem in os.listdir(path) if elem.endswith(naming_flag)]
            return dict(zip(cameras, filepaths)) 
                                 
    def _iterate_template_matching(self, camera: str, image: str, level: str, object_of_interest: str) -> (str, (int, int)):
        # use template matching function to perform template matching for each level
        
        self.template_matched = False
        template = cv2.imread(self.cameras_and_paths_template[camera][object_of_interest][level], 0)
        
        accuracies_and_locations = {}
        
        for method in self.tm_methods:
            # test different template matching methods and select the best one
            accuracy, location = self._match_template(image, template, method)
            accuracies_and_locations[accuracy] = location
        best_accuracy = max(accuracies_and_locations.keys())
        self._evaluate_template_matching(best_accuracy)
            
        if self.template_matched:
            # optional: visualize template matching 
            if self.visualize_matching:
                self._visualize_template_matching(image, template, location = accuracies_and_locations[best_accuracy])
            return template, location
                    
        if self.template_matched == False:
            print('template is not fit for template matching! Use .create_template to create a new template for this object:', object_of_interest, camera)
            self.template_matching_ongoing = False
    
    
    def _match_template(self, image: str, template: str, method: str) -> (int, (int, int)):
        # matches template to image and returns accuracy and location of top left corner 
        img = image.copy()
        result = cv2.matchTemplate(image = img, templ = template, method = method)
        min_value, max_value, min_location, max_location = cv2.minMaxLoc(result)
        
        if method == cv2.TM_SQDIFF_NORMED:
            # the normed-/sqdiff methods have the position of the matched template in the min location
            location = min_location
            value = min_value
            accuracy = 1-abs(0-value)
        else:
            # All other methods show the matched template position in the max location
            location = max_location
            value = max_value
            accuracy = value
        return  accuracy, location
        
        
    def _visualize_template_matching(self, image: str, template: str, location: (int)):
        height_template, width_template = template.shape
        bottom_right_corner = (location[0] + width_template, location[1] + height_template)
        cv2.rectangle(image, location, bottom_right_corner, color = 0)
        cv2.imshow('Template Matched Image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows
        
    def _evaluate_template_matching(self, accuracy: int) -> bool:
        if accuracy > self.template_matching_threshold:
            print('Template matching worked with a ' + str(accuracy) + ' accuracy')
            self.template_matched = True
        else:
             print('Template matching is not ideal, now trying a different method!')
    
    def _get_mean_coordinates(self, image: str, template: str, location: (int, int)) ->(int, int):
        # get coordinates for input image
        height_template, width_template = template.shape
        center_coordinates =location[0] + width_template/2, location[1] + height_template/2        
        return center_coordinates
    
    def _transpose_coordinates(self, coordinates: (int, int), ofset: (int, int)) ->(int, int):
        # transpose coordinates from image slice to original image
        return (coordinates[0] + ofset[0], coordinates[1] + ofset[1])
    
    def _check_whether_analyzed(self):
        # Check for pickle file --> has this data already been analyzed? CURRENTLY NON FUNCTIONAL
        file_exists = os.path.exists(self.directory_positions_jpg + 'object_coordinates.pickle')
        if file_exists:            
            inputquesiton = input('This data has already been analysed! \n\
            Do you really want to reanalyse the data? This would overwrite the old coordinates\
            type "yes" or "no"')
            if inputquesiton:
                return True
            
        return False

In [ ]:
path220824 = 'C:\\Users\\MS\\Documents\\Studium\\OpeningTrack\\coding\\test_data\\positions\\220824\\'
path220825 = 'C:\\Users\\MS\\Documents\\Studium\\OpeningTrack\\coding\\test_data\\positions\\220825\\'
path220826 = 'C:\\Users\\MS\\Documents\\Studium\\OpeningTrack\\coding\\test_data\\positions\\220826\\'

In [ ]:
day220824 = TemplateMatching(path220824, visualize_matching= True)
day220825 = TemplateMatching(path220825)
day220826 = TemplateMatching(path220826)

In [ ]:
day220824.run()

In [ ]:
day220825.run()

In [ ]:
day220826.run()

In [ ]:
day220824.object_coordinates_per_camera


# Sanity check

In [ ]:
for day in [day220824, day220825, day220826]:
    for cam in day.object_coordinates_per_camera.keys():
        for obj in day.object_coordinates_per_camera[cam].keys():
            image_temp = cv2.imread(day.cameras_and_paths_positions[cam])
            coord1 = int(day.object_coordinates_per_camera[cam][obj][0])
            coord2 = int(day.object_coordinates_per_camera[cam][obj][1])
            cv2.circle(image_temp, (coord1, coord2), 2, (0,0,255), -1)
            cv2.imshow('Sanity check for ' + str(cam) + ' ' + str(obj), image_temp)
            cv2.waitKey(0)
            cv2.destroyAllWindows

Future notes:
- would be great to use the same template for specific objects, like all screws, all backcorners, ...
    However, it then is basically a matter of making the code more complicated instead the template folder. I doubt that any option is prefereable here.
- build more templates
- Switch to scikit for template matching, as open cv does not perform as well.

Sanity check failed on 220824 Bottom screw1+screw2 and 220826 Side2 screw1. not great.

# Code graveyard 

In [ ]:
# Different, nonfunctional solution for generating template dictionaries
for filepath in filepaths:
    if filepath.endswith(naming_flag):
        split_name = filepath.split("_")
        camera, objecttofind, level = split_name[-4], split_name[-3], split_name[-2]
        cameras, objects, levels = [], [], []
        cameras.append(camera)
        objects.append(objecttofind)
        levels.append(level)